<a href="https://colab.research.google.com/github/ravitejalakkoju/covid-19-prediction-fyp/blob/main/Covid_19_Detection_Categorical.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opendatasets --upgrade --quiet

In [ ]:
import opendatasets as od
from sklearn.model_selection import train_test_split
import os
import matplotlib.pyplot as plt
%matplotlib inline
from torchvision.datasets import ImageFolder
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds
import pathlib

In [ ]:
kaggle_dataset_url="https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia"

In [ ]:
github_dataset_url="https://github.com/education454/datasets.git"

In [ ]:
od.download(kaggle_dataset_url)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: 

In [ ]:
!git clone https://github.com/education454/datasets.git

In [ ]:
kaggle_dir='./chest-xray-pneumonia/chest_xray'
ktrain_dir = os.path.join(kaggle_dir, 'train')
ktest_dir = os.path.join(kaggle_dir, 'test')
kval_dir = os.path.join(kaggle_dir, 'val')
train_pneumonia_dir = os.path.join(ktrain_dir, 'PNEUMONIA')
train_normal_dir = os.path.join(ktrain_dir, 'NORMAL')
test_pneumonia_dir = os.path.join(ktest_dir, 'PNEUMONIA')
test_normal_dir = os.path.join(ktest_dir, 'NORMAL')
val_pneumonia_dir = os.path.join(kval_dir, 'PNEUMONIA')
val_normal_dir = os.path.join(kval_dir, 'NORMAL')

In [ ]:
github_dir = './datasets/Data'
gtrain_dir = os.path.join(github_dir, 'train')
gtest_dir = os.path.join(github_dir, 'test')
train_covid_dir = os.path.join(gtrain_dir, 'COVID19')
test_covid_dir = os.path.join(gtest_dir, 'COVID19')

In [ ]:
ktrain_dir = pathlib.Path(ktrain_dir)
ktest_dir = pathlib.Path(ktest_dir)
kval_dir = pathlib.Path(kval_dir)
print("TOTAL: %3d, PNEUMONIA: %3d, NORMAL: %3d" % (len(list(ktrain_dir.glob('*/*'))), len(list(ktrain_dir.glob('*PNEUMONIA/*'))), len(list(ktrain_dir.glob('*NORMAL/*')))))
print("TOTAL: %3d, PNEUMONIA: %3d, NORMAL: %3d" % (len(list(ktest_dir.glob('*/*'))), len(list(ktest_dir.glob('*PNEUMONIA/*'))), len(list(ktest_dir.glob('*NORMAL/*')))))
print("TOTAL: %3d, PNEUMONIA: %3d, NORMAL: %3d" % (len(list(kval_dir.glob('*/*'))), len(list(kval_dir.glob('*PNEUMONIA/*'))), len(list(kval_dir.glob('*NORMAL/*')))))

In [ ]:
gtrain_dir = pathlib.Path(gtrain_dir)
gtest_dir = pathlib.Path(gtest_dir)
print("TOTAL: %3d, COVID19: %3d, NORMAL: %3d" % (len(list(gtrain_dir.glob('*/*'))), len(list(gtrain_dir.glob('*COVID19/*'))), len(list(gtrain_dir.glob('*NORMAL/*')))))
print("TOTAL: %3d, COVID19: %3d, NORMAL: %3d" % (len(list(gtest_dir.glob('*/*'))), len(list(gtest_dir.glob('*COVID19/*'))), len(list(gtest_dir.glob('*NORMAL/*')))))

In [ ]:
train_pneumonia_names = os.listdir(train_pneumonia_dir)
print(train_pneumonia_names[:10])

train_covid_names = os.listdir(train_covid_dir)
print(train_covid_names[:10])

train_normal_names = os.listdir(train_normal_dir)
print(train_normal_names[:10])

test_pneumonia_names = os.listdir(test_pneumonia_dir)
print(train_pneumonia_names[:10])

test_covid_names = os.listdir(test_covid_dir)
print(train_covid_names[:10])

test_normal_names = os.listdir(test_normal_dir)
print(train_normal_names[:10])

In [ ]:
import matplotlib.image as mpimg

nrows = 4
ncols = 4

fig = plt.gcf()
fig.set_size_inches(12,12)

next_covid_pic = [os.path.join(train_covid_dir, fname)for fname in train_covid_names[0:4]]
next_pneumonia_pic = [os.path.join(train_pneumonia_dir, fname)for fname in train_pneumonia_names[0:4]]
next_normal_pic = [os.path.join(train_normal_dir, fname)for fname in train_normal_names[0:4]]

print(next_covid_pic)
print(next_pneumonia_pic)
print(next_normal_pic)

for i , img_path in enumerate(next_covid_pic + next_pneumonia_pic + next_normal_pic) : 
  data = img_path.split('/', 5)[5]
  sp = plt.subplot(nrows, ncols,i+1)
  sp.axis('Off')
  img = mpimg.imread(img_path)
  sp.set_title(data, fontsize = 10)
  plt.imshow(img, cmap = 'gray')

In [ ]:
categorical_test_df = pd.DataFrame(columns =['class', 'o_directory', 'i_directory'])
categorical_train_df = pd.DataFrame(columns =['class', 'o_directory', 'i_directory'])
for dirpath, dirnames, filenames in os.walk(train_normal_dir):
    i = 0;
    for filename in filenames:
        categorical_train_df.loc[-1] = ["NORMAL", ('NORMAL/' + filename), (train_normal_dir + '/' +filename) ] 
        categorical_train_df.index = categorical_train_df.index + 1 
        categorical_train_df = categorical_train_df.sort_index()
        i += 1
        if i==545:
            break
for dirpath, dirnames, filenames in os.walk(test_normal_dir):
    i = 0;
    for filename in filenames:
        categorical_test_df.loc[-1] = ["NORMAL", ('NORMAL/' + filename), (test_normal_dir + '/' +filename)] 
        categorical_test_df.index = categorical_test_df.index + 1 
        categorical_test_df = categorical_test_df.sort_index() 
        i += 1
        if i==80:
            break 
for dirpath, dirnames, filenames in os.walk(train_covid_dir):
    i = 0;
    for filename in filenames:
        categorical_train_df.loc[-1] = ["COVID19", ('COVID19/' + filename), (train_covid_dir + '/' +filename)] 
        categorical_train_df.index = categorical_train_df.index + 1 
        categorical_train_df = categorical_train_df.sort_index() 
for dirpath, dirnames, filenames in os.walk(test_covid_dir):
    i = 0;
    for filename in filenames:
        categorical_test_df.loc[-1] = ["COVID19", ('COVID19/' + filename), (test_covid_dir + '/' +filename)] 
        categorical_test_df.index = categorical_test_df.index + 1 
        categorical_test_df = categorical_test_df.sort_index()
        i += 1
        if i==80:
            break 
for dirpath, dirnames, filenames in os.walk(train_pneumonia_dir):
    i = 0;
    for filename in filenames:
        categorical_train_df.loc[-1] = ["PNEUMONIA", ('PNEUMONIA/' + filename), (train_pneumonia_dir + '/' +filename)] 
        categorical_train_df.index = categorical_train_df.index + 1 
        categorical_train_df = categorical_train_df.sort_index() 
        i += 1
        if i==545:
            break
for dirpath, dirnames, filenames in os.walk(test_pneumonia_dir):
    i = 0;
    for filename in filenames:
        categorical_test_df.loc[-1] = ["PNEUMONIA", ('PNEUMONIA/' + filename), (test_pneumonia_dir + '/' +filename)] 
        categorical_test_df.index = categorical_test_df.index + 1 
        categorical_test_df = categorical_test_df.sort_index()
        i += 1
        if i==80:
            break 

In [ ]:
categorical_test_df = pd.DataFrame(columns =['class', 'o_directory', 'i_directory'])
categorical_train_df = pd.DataFrame(columns =['class', 'o_directory', 'i_directory'])
for dirpath, dirnames, filenames in os.walk(train_normal_dir):
    for filename in filenames:
        categorical_train_df.loc[-1] = ["NORMAL", ('NORMAL/' + filename), (train_normal_dir + '/' +filename) ] 
        categorical_train_df.index = categorical_train_df.index + 1 
        categorical_train_df = categorical_train_df.sort_index()
for dirpath, dirnames, filenames in os.walk(test_normal_dir):
    for filename in filenames:
        categorical_test_df.loc[-1] = ["NORMAL", ('NORMAL/' + filename), (test_normal_dir + '/' +filename)] 
        categorical_test_df.index = categorical_test_df.index + 1 
        categorical_test_df = categorical_test_df.sort_index()  
for dirpath, dirnames, filenames in os.walk(val_normal_dir):
    for filename in filenames:
        categorical_test_df.loc[-1] = ["NORMAL", ('NORMAL/' + filename), (val_normal_dir + '/' +filename)] 
        categorical_test_df.index = categorical_test_df.index + 1 
        categorical_test_df = categorical_test_df.sort_index()  
for dirpath, dirnames, filenames in os.walk(train_covid_dir):
    for filename in filenames:
        categorical_train_df.loc[-1] = ["COVID19", ('COVID19/' + filename), (train_covid_dir + '/' +filename)] 
        categorical_train_df.index = categorical_train_df.index + 1 
        categorical_train_df = categorical_train_df.sort_index() 
for dirpath, dirnames, filenames in os.walk(test_covid_dir):
    for filename in filenames:
        categorical_test_df.loc[-1] = ["COVID19", ('COVID19/' + filename), (test_covid_dir + '/' +filename)] 
        categorical_test_df.index = categorical_test_df.index + 1 
        categorical_test_df = categorical_test_df.sort_index() 
for dirpath, dirnames, filenames in os.walk(train_pneumonia_dir):
    for filename in filenames:
        categorical_train_df.loc[-1] = ["PNEUMONIA", ('PNEUMONIA/' + filename), (train_pneumonia_dir + '/' +filename)] 
        categorical_train_df.index = categorical_train_df.index + 1 
        categorical_train_df = categorical_train_df.sort_index() 
for dirpath, dirnames, filenames in os.walk(test_pneumonia_dir):
    for filename in filenames:
        categorical_test_df.loc[-1] = ["PNEUMONIA", ('PNEUMONIA/' + filename), (test_pneumonia_dir + '/' +filename)] 
        categorical_test_df.index = categorical_test_df.index + 1 
        categorical_test_df = categorical_test_df.sort_index() 
for dirpath, dirnames, filenames in os.walk(val_pneumonia_dir):
    for filename in filenames:
        categorical_test_df.loc[-1] = ["PNEUMONIA", ('PNEUMONIA/' + filename), (val_pneumonia_dir + '/' +filename)] 
        categorical_test_df.index = categorical_test_df.index + 1 
        categorical_test_df = categorical_test_df.sort_index() 

In [ ]:
categorical_train_df

In [ ]:
categorical_test_df

In [ ]:
def copy_images_categorical(df, directory):
    
    # input and output directory
    output_path = "CategoricalDataSet/" + directory

    # remove all files from previous fold
    if os.path.exists(output_path):
        shutil.rmtree(output_path)

    # create folder for files from this fold
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    # create subfolders for each class
    classs=['NORMAL','COVID19','PNEUMONIA']
    for c in classs:
        if not os.path.exists(output_path + '/' + c):
            os.makedirs(output_path + '/' + c)
        
    # copy files
    for i, row in df.iterrows():
        path_from = row['i_directory']
        path_to = "{}/{}".format(output_path, (row['i_directory'].split('/')[-2]  + '/' + row['i_directory'].split('/')[-1]))
        shutil.copy(path_from, path_to)

In [ ]:
copy_images_categorical(categorical_test_df, 'test')
copy_images_categorical(categorical_train_df, 'train')

In [ ]:
print(len(os.listdir('./CategoricalDataSet/test/NORMAL')))
print(len(os.listdir('./CategoricalDataSet/test/COVID19')))
print(len(os.listdir('./CategoricalDataSet/test/PNEUMONIA')))

In [ ]:
print(len(os.listdir('./CategoricalDataSet/train/NORMAL')))
print(len(os.listdir('./CategoricalDataSet/train/COVID19')))
print(len(os.listdir('./CategoricalDataSet/train/PNEUMONIA')))

In [ ]:
train_dir = './CategoricalDataSet/train'
test_dir = './CategoricalDataSet/test'

In [ ]:
train_dir = pathlib.Path(train_dir)
image_count = len(list(train_dir.glob('*/*')))
print(image_count)

In [ ]:
test_dir = pathlib.Path(test_dir)
image_count = len(list(test_dir.glob('*/*')))
print(image_count)

In [ ]:
train_dir = pathlib.Path(train_dir)
test_dir = pathlib.Path(test_dir)
print("TOTAL: %3d, PNEUMONIA: %3d, COVID19: %3d, NORMAL: %3d" % (len(list(train_dir.glob('*/*'))), len(list(train_dir.glob('*PNEUMONIA/*'))), len(list(train_dir.glob('*COVID19/*'))), len(list(train_dir.glob('*NORMAL/*')))))
print("TOTAL: %3d, PNEUMONIA: %3d, COVID19: %3d, NORMAL: %3d" % (len(list(test_dir.glob('*/*'))), len(list(test_dir.glob('*PNEUMONIA/*'))), len(list(test_dir.glob('*COVID19/*'))), len(list(test_dir.glob('*NORMAL/*')))))

In [ ]:
image_size = (512, 512)

In [ ]:
train_data_gen = ImageDataGenerator(rescale  = 1./255, 
                                    validation_split=0.1)

#                                     horizontal_flip = True,
#                                     zoom_range = 0.2,
#                                     rotation_range = 0,
#                                     fill_mode='nearest'


test_data_gen = ImageDataGenerator(rescale  = 1./255)


train_generator = train_data_gen.flow_from_directory(train_dir,
                                                     target_size = image_size,
                                                     subset = 'training', 
                                                     batch_size = 32,
                                                     class_mode = 'categorical')

val_generator = train_data_gen.flow_from_directory(train_dir,
                                                     target_size = image_size,
                                                     subset = 'validation', 
                                                     batch_size = 32,
                                                     class_mode = 'categorical')

test_generator = test_data_gen.flow_from_directory(test_dir,
                                                     target_size = image_size,
                                                     batch_size = 32,
                                                     class_mode = 'categorical')

In [ ]:
class_names = os.listdir(train_dir)

In [ ]:
from matplotlib import pyplot

for i in range(15):
  pyplot.subplot(4, 4, i + 1)
  img, labels = train_generator.next()
  image = img[0]
  pyplot.xticks([])
  pyplot.yticks([])
  pyplot.subplots_adjust(top=1, left=0.5)
  pyplot.imshow(image[:, :, 0], cmap='gray')
  pyplot.title(class_names[labels[i].argmax()])
  pyplot.gcf().set_size_inches(20, 10)

pyplot.show()

Model Formulation

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical

In [ ]:
model = Sequential()
model.add(Conv2D(filters=64, kernel_size=(5, 5), activation="relu", input_shape=(512,512,1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Conv2D(filters=32, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.metrics import AUC
from tensorflow import keras
from keras.optimizers import Adam

In [ ]:
model.compile(Adam(lr=0.001), loss = 'categorical_crossentropy', metrics = ['accuracy', keras.metrics.AUC(), keras.metrics.Precision(), keras.metrics.Recall()])

In [ ]:
model_fit = model.fit(train_generator, epochs=2, validation_data=val_generator, batch_size=96)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import itertools 

In [ ]:
test_loss, test_acc, test_auc, test_precision, test_recall = model.evaluate(test_generator)
print('test_acc : {} test_loss : {}, test_auc: {}, test_precision: {}, test_precision: {}'.format(test_acc, test_loss, test_auc, test_precision, test_recall))

In [ ]:
f1_score(0.8163001537322998, 0.8163001537322998)

In [ ]:
Y_pred = model.predict(test_generator)

In [ ]:
y_pred = np.argmax(Y_pred, axis = 1)

print('Confusion Matrix')

cm = confusion_matrix(test_generator.classes, y_pred)

print(cm)

In [ ]:
target_names = []

for key in train_generator.class_indices:

    target_names.append(key)

In [ ]:
print('Classification Report')

print(classification_report(test_generator.classes, y_pred, target_names=target_names))

In [ ]:
def f1_score(test_precision, test_recall): 
  return ((2* test_precision * test_recall) / (test_precision + test_recall))

In [ ]:
from tensorflow import keras
from keras import models
from keras.models import Model
from keras.models import Sequential
from keras import layers
from keras.applications import VGG16
from keras.applications.densenet import DenseNet121
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_v3 import InceptionV3
from keras import optimizers
from keras.layers.core import Flatten, Dense, Dropout, Lambda

In [ ]:
conv_base = DenseNet121(weights='imagenet',
 include_top=False,
 input_shape=(512, 512, 3))
conv_base.trainable = False
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy',
 optimizer=optimizers.Adam(lr=0.001),
 metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall(), keras.metrics.AUC()])

In [ ]:
conv_base = ResNet50(weights='imagenet',
 include_top=False,
 input_shape=(512, 512, 3))
conv_base.trainable = False
model2 = models.Sequential()
model2.add(conv_base)
model2.add(layers.Flatten())
model2.add(layers.Dense(256, activation='relu'))
model2.add(layers.Dense(3, activation='softmax'))
model2.compile(loss='categorical_crossentropy',
 optimizer=optimizers.Adam(lr=0.001),
 metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall(), keras.metrics.AUC()])

In [ ]:
conv_base = VGG16(weights='imagenet',
 include_top=False,
 input_shape=(512, 512, 3))
conv_base.trainable = False
model3 = models.Sequential()
model3.add(conv_base)
model3.add(layers.Flatten())
model3.add(layers.Dense(256, activation='relu'))
model3.add(layers.Dense(3, activation='softmax'))
model3.compile(loss='categorical_crossentropy',
 optimizer=optimizers.Adam(lr=0.001),
 metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall(), keras.metrics.AUC()])

In [ ]:
conv_base = InceptionV3(weights='imagenet',
 include_top=False,
 input_shape=(512, 512, 3))
conv_base.trainable = False
model4 = models.Sequential()
model4.add(conv_base)
model4.add(layers.Flatten())
model4.add(layers.Dense(256, activation='relu'))
model4.add(layers.Dense(3, activation='softmax'))
model4.compile(loss='categorical_crossentropy',
 optimizer=optimizers.Adam(lr=0.001),
 metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall(), keras.metrics.AUC()])

In [ ]:
from keras.applications import InceptionResNetV2

In [ ]:
conv_base = InceptionV3(weights='imagenet',
 include_top=False,
 input_shape=(512, 512, 3))
conv_base.trainable = False
model5 = models.Sequential()
model5.add(conv_base)
model5.add(layers.Flatten())
model5.add(layers.Dense(256, activation='relu'))
model5.add(layers.Dense(3, activation='softmax'))
model5.compile(loss='categorical_crossentropy',
 optimizer=optimizers.Adam(lr=0.001),
 metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall(), keras.metrics.AUC()])

In [ ]:
model_fit = model.fit(train_generator, epochs=2, validation_data=val_generator, batch_size=96)

In [ ]:
test_loss, test_acc, test_precision, test_recall, test_auc = model.evaluate(test_generator)
print('test_acc : {} test_loss : {} test_precision: {}, test_recall: {}'.format(test_acc, test_loss, test_precision, test_recall))

In [ ]:
model_fit2 = model2.fit(train_generator, epochs=3, validation_data=val_generator, batch_size=96)

In [ ]:
test_loss2, test_acc2, test_precision2, test_recall2, test_auc2 = model2.evaluate(test_generator)
print('test_acc : {} test_loss : {} test_precision: {}, test_recall: {}'.format(test_acc2, test_loss2, test_precision2, test_recall2))

In [ ]:
print(f1_score(test_precision2, test_recall2))

In [ ]:
model_fit3 = model3.fit(train_generator, epochs=3, validation_data=val_generator, batch_size=96)

In [ ]:
test_loss3, test_acc3, test_precision3, test_recall3, test_auc3 = model3.evaluate(test_generator)
print('test_acc : {} test_loss : {} test_precision: {}, test_recall: {}'.format(test_acc3, test_loss3, test_precision3, test_recall3))

In [ ]:
print(f1_score(test_precision3, test_recall3))

In [ ]:
model_fit4 = model4.fit(train_generator, epochs=3, validation_data=val_generator, batch_size=96)

In [ ]:
test_loss4, test_acc4, test_precision4, test_recall4, test_auc4 = model4.evaluate(test_generator)
print('test_acc : {} test_loss : {} test_precision: {}, test_recall: {}'.format(test_acc4, test_loss4, test_precision4, test_recall4))

In [ ]:
print(f1_score(test_precision4, test_recall4))

In [ ]:
model_fit5 = model5.fit(train_generator, epochs=3, validation_data=val_generator, batch_size=96)

In [ ]:
test_loss5, test_acc5, test_precision5, test_recall5, test_auc5 = model5.evaluate(test_generator)
print('test_acc : {} test_loss : {} test_precision: {}, test_recall: {}'.format(test_acc5, test_loss5, test_precision5, test_recall5))

In [ ]:
print('test_acc : {} test_loss : {} test_precision: {}, test_recall: {}'.format(test_acc5, test_loss5, test_precision5, test_recall5))


In [ ]:
print(test_auc5)

In [ ]:
print(f1_score(test_precision5, test_recall5))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import itertools 

In [ ]:
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):

    """

    This function prints and plots the confusion matrix.

    Normalization can be applied by setting `normalize=True`.

    """

    plt.figure(figsize=(10,10))



    plt.imshow(cm, interpolation='nearest', cmap=cmap)

    plt.title(title)

    plt.colorbar()



    tick_marks = np.arange(len(classes))

    plt.xticks(tick_marks, classes, rotation=45)

    plt.yticks(tick_marks, classes)



    if normalize:

        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

        cm = np.around(cm, decimals=2)

        cm[np.isnan(cm)] = 0.0

        print("Normalized confusion matrix")

    else:

        print('Confusion matrix, without normalization')

    thresh = cm.max() / 2.

    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):

        plt.text(j, i, cm[i, j],

                 horizontalalignment="center",

                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()

    plt.ylabel('True label')

    plt.xlabel('Predicted label')

In [ ]:
target_names = []

for key in train_generator.class_indices:

    target_names.append(key)

In [ ]:
Y_pred = model.predict(test_generator)

In [ ]:
y_pred = np.argmax(Y_pred, axis = 1)

print('Confusion Matrix')

cm = confusion_matrix(test_generator.classes, y_pred)

print(cm)
plot_confusion_matrix(cm, target_names, title='Confusion Matrix')

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
langs = ['Covid-19', 'Normal', 'Pneumonia']
students = [len(list(train_dir.glob('*COVID19/*'))), len(list(train_dir.glob('*NORMAL/*'))), len(list(train_dir.glob('*PNEUMONIA/*')))]
ax.bar(langs,students)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
langs = ['Covid-19', 'Normal', 'Pneumonia']
students = [len(list(test_dir.glob('*COVID19/*'))), len(list(test_dir.glob('*NORMAL/*'))), len(list(test_dir.glob('*PNEUMONIA/*')))]
ax.bar(langs,students)
plt.show()